**Importing Libraries**

In [ ]:
import pandas as pd

pd.set_option("display.max_columns", None)
pd.set_option("display.width", 30)
pd.set_option("display.expand_frame_repr", False)

**Reading Datasets**

In [ ]:
books = pd.read_csv("/kaggle/input/book-recommendation-dataset/Books.csv")
ratings = pd.read_csv("/kaggle/input/book-recommendation-dataset/Ratings.csv")
users = pd.read_csv("/kaggle/input/book-recommendation-dataset/Users.csv")
books.head()

In [ ]:
books.shape

In [ ]:
ratings.head()

In [ ]:
users.head()

In [ ]:
users.shape

We merge books and ratings datasets on the 'ISBN' columns. We also merge ratings and users datasets on 'User-ID' columns.

In [ ]:
df_ = ratings.merge(books, on= 'ISBN')
df_ = df_.merge(users, on= 'User-ID')
df_ = df_[["User-ID", "Book-Rating", "Book-Title"]]
df = df_.copy()

In [ ]:
df.columns = ["users", "ratings", "titles"]
df.head()

Let's see how many times which book has been rated.

In [ ]:
rating_count = pd.DataFrame(df["titles"].value_counts())
rating_count

We separate books with more than 50 votes as common books.

In [ ]:
common_books = rating_count[rating_count["count"] > 100].index

In [ ]:
#users_books = df.loc[df['titles'].isin(common_books)]

We are preparing our pivot table on which we will perform operations. The difference from the previous one is that the indexes consist of users.

In [ ]:
users_books = df.pivot_table(index= 'users', columns= 'titles', values= 'ratings')
users_books

Now, we choose a random user to make a recommendation.

In [ ]:
random_user = int(pd.Series(df['user']).sample(1, random_state= 19).values)
random_user

We create a list of books read by our Random user.

In [ ]:
random_user_df = users_books.loc[users_books.index == random_user]
books_readed = random_user_df.columns[random_user_df.notna().any()].tolist()
books_readed_df = users_books.loc[users_books['user'] == random_user, users_books.columns.isin(books_readed)]

We find out how many of the books other users have read that the user we selected has read.

In [ ]:
count_books = count_books[books_readed]
count_books = count_books.T.notnull().sum()
count_books #indexlerde user id'ler var

We change our columns and names.

In [ ]:
count_books = count_books.reset_index()
count_books.columns = ['user', 'books_count']

We find users who read 60 percent or more of the books our user reads.

In [ ]:
user_same_books = count_books[count_books['books_count'] > len(books_readed)*60/100, 'user']

In [ ]:
users_readed = users_books.loc[users_books.index.isin(user_same_books), books_readed]
users_readed

We find the correlation between users and create the dataframe.

In [ ]:
corr_df = users_readed.T.corr().unstack().sort_values(ascending=False).drop_duplicates()
corr_df.index.names = ['User1', 'User2']
corr_df = pd.DataFrame(corr_df, columns= ['corr'])
corr_df = corr_df.reset_index()
corr_df

We bring our selected user to the first row.

In [ ]:
first_row = corr_df[corr_df['User1'] == random_user]
other_rows = corr_df[corr_df['User1'] != random_user]
corr_df = pd.concat([first_row, other_rows])

corr_df.loc[corr_df['User1'] == random_user]

We find the most highly correlated users.

In [ ]:
top_users = corr_df.loc[corr_df['User1'] == random_user,
                        corr_df['corr'] > 0.60]

top_users

We merge these users with our first dataframe.

In [ ]:
top_users_df = top_users.merge(df, left_on='User2', right_on='users', how='right')

We define a new variable called weighted rating.

Thanks to this variable, we can make the best recommendations to the user we choose.

In [ ]:
top_users_df['weighted_rating'] = top_users_df['corr'] * top_users_df['rating']

We create a recommendation dataframe based on the mean of our weighted rating.

In [ ]:
recommendation_df = pd.DataFrame(top_users_df.groupby['titles']['weighted_rating'].mean())
recommendation_df

In [ ]:
recommendation_df = recommendation_df.loc[recommendation_df['weighted_rating'] > 3.5].sort_values(ascending=False)
#recommendation_df = recommendation_df.loc[recommendation_df['weighted_rating'] > 3.5].sort_values(ascending=False).iloc[0:5]

In [ ]:
#bunu tam çıkaramadım. Tekrar bakmam gerek.
recommend_books = df.loc[df['titles'].isin(recommendation_df.index), 'titles']